In [1]:
from transformers import pipeline

model_id = "cjvt/GaMS-2B-Instruct"

pline = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto" # "auto" is to be used on google colab
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/288 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

**QUESTION ANSWERING**

In [3]:
# Asking a question in slovenian and observing the response generation
message = [{"role": "user", "content": "Kdaj se je žgodil padec berlinskega zidu?"}]
response = pline(message, max_new_tokens=512)
print("Model's response:", response[0]["generated_text"][-1]["content"])

# Asking a question in english and observing the response generation
message = [{"role": "user", "content": "When did the Berlin wall fall?"}]
response = pline(message, max_new_tokens=512)
print("Model's response:", response[0]["generated_text"][-1]["content"])

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Model's response: Padec berlinskega zidu se je zgodil 9. novembra 1989.

Model's response: On November 9, 1989



**TRANSLATION**

In [4]:
# Asking the model to translate from slovenian to english
message = [{"role": "user", "content": "Translate this sentence from slovenian to english : Vlada Republike Slovenije je organ izvršilne oblasti, hkrati pa je tudi najvišji organ državne uprave."}]
response = pline(message, max_new_tokens=512)
print("Model's response:", response[0]["generated_text"][-1]["content"])

# Asking the model to translate from slovenian to french
message = [{"role": "user", "content": "Translate this sentence from slovenian to french : Vlada Republike Slovenije je organ izvršilne oblasti, hkrati pa je tudi najvišji organ državne uprave."}]
response = pline(message, max_new_tokens=512)
print("Model's response:", response[0]["generated_text"][-1]["content"])


Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Model's response: The Government of the Republic of Slovenia is an organ of the executive power and at the same time the highest organ of state administration.

Model's response: La Vlade de la Republique de Slovaquie est le pouvoir exécutif et il est le plus haut organe de l'administration publique.



**SUMMARIZING**

In [13]:
import requests
from bs4 import BeautifulSoup

In [14]:
# Download the article
def fetch_article_text(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    paragraphs = soup.find_all("p")
    return "\n".join(p.get_text() for p in paragraphs)

url = "https://www.bbc.com/news/articles/ce3ex92557jo" #change the url here to analyze antoher article
article_text = fetch_article_text(url)

# Summary
new_message = response[0]["generated_text"]
new_message.append({
    "role": "user",
    "content": "Please summarize the article above using bullet points." # there we can precise our demands to the model
})

response = pline(new_message, max_new_tokens=512)
print("\n=== Summary ===\n")
print(response[0]["generated_text"][-1]["content"])


Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Summary ===

Here is a summary of the article:

1. Elon Musk's X is under investigation for potential data extraction and child pornography crimes.

2. The investigation is part of a broader probe into the controversial AI chatbot Grok.

3. Musk and former CEO Linda Yaccarino have been summoned to court hearings.

4. X has denied the allegations, characterizing the investigation as an attack on free speech.

5. Concerns about child pornography and AI-generated images have led to public pressure on X.

6. The company has implemented controls to mitigate the issue of AI-generated child pornography.

7. The European Commission and UK regulators are investigating X over its practices.

8. The town of Marazion has banned X from using its broadband network due to concerns about Grok.

9. SpaceX, Elon Musk's company, is involved in Starlink, a satellite internet service.

10. Starlink has been linked to recent Russian attacks on Ukraine, particularly using drones.

11. SpaceX's plans for

**MODIFYING SOME PARAMETERS FOR TRANSLATING AND SUMMARIZING**

In [ ]:
# Asking the model to translate from slovenian to english with new parameters
message = [{"role": "user", "content": "Translate this sentence from slovenian to english : Vlada Republike Slovenije je organ izvršilne oblasti, hkrati pa je tudi najvišji organ državne uprave."}]
response = pline(
    message,
    max_new_tokens=1000,  #this value can be an int
    temperature=0.8,      #you can play around with this value, WARNING it has to be a float "2.0" and not an int "2"
    top_p=0.3,            #has to be < to 1 (and to be a float)
    do_sample=True,
    max_length=None
)
print("Model's response:", response[0]["generated_text"][-1]["content"])

Passing `generation_config` together with generation-related arguments=({'temperature', 'do_sample', 'top_p', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=1000) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Model's response: The Government of the Republic of Slovenia is an organ of executive power and at the same time the highest organ of the state administration.



In [16]:
# Summarizing the article with new parameters
# Download the article using the url
def fetch_article_text(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    paragraphs = soup.find_all("p")
    return "\n".join(p.get_text() for p in paragraphs)

url = "https://www.bbc.com/news/articles/ce3ex92557jo" #change the url here to analyze antoher article
article_text = fetch_article_text(url)

# Summary
new_message = response[0]["generated_text"]
new_message.append({
    "role": "user",
    "content": "Please summarize the article above using bullet points." # there we can precise our demands to the model
})

response = pline(new_message, max_new_tokens=512)
print("\n=== Summary with new params ===\n")
response = pline(
    message,
    max_new_tokens=1000,  #this value can be an int
    temperature=0.8,      #you can play around with this value, WARNING it has to be a float "2.0" and not an int "2"
    top_p=0.3,            #has to be < to 1 (and to be a float)
    do_sample=True,
    max_length=None
)
print(response[0]["generated_text"][-1]["content"])

Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1000) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Summary with new params ===

The French offices of Elon Musk's X have been raided by the Paris prosecutor's cyber-crime unit as part of an investigation into suspected offences including unlawful data extraction and complicity in the possession of child pornography.
The prosecutor's office said the probe began in January 2025 when it started looking into content recommended by the social media platform's algorithm, before being widened to include its controversial AI chatbot, Grok.
Elon Musk and former X chief executive officer Linda Yaccarino have been summoned to appear at hearings in April.
X has yet to comment but has previously characterised the investigation as an attack on free speech.
The investigation is now widening to include multiple areas, including complicity in possession or organised distribution of images of children of a pornographic nature, infringement of people's image rights with sexual deepfakes, and fraudulent data extraction by an organised group.
The pros